# Random velocity gradient

Random velocity gradient sampled from standard distribution, followed by relaxation. The velocity gradient is normalised. The Henky strain `H` is the duration of the velocity gradient event. 

In [2]:
import numpy as np
from scipy.interpolate import interp1d
import pywt

from ipywidgets import interactive
import ipywidgets as widgets

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 8]

output_file = 'inputs/random/scenario_random'

# Number of independent scenarios
n_scenarios = 5

# Henky strain
H = 5

## Sample velocity gradient data

In [18]:
times = [[0,H,H+1]]*n_scenarios
gradUs = []

for i in range(n_scenarios):
    gradU = np.random.standard_normal(size=(3,3))
    gradU = gradU - (gradU[0,0] + gradU[1,1] + gradU[2,2])/3*np.eye(3)
    gradUs.append([gradU, np.zeros((3,3)), np.zeros((3,3))])

## Analyse strain

In [19]:
gradUs_norm = []

for gradU in gradUs:

    # Get principal values for the strain
    D = 0.5*(gradU + np.transpose(gradU, axes=(0,2,1)))
    strains = np.linalg.eigvalsh(D)
    max_Wi = np.max(strains[:,2])
    
    # Normalize
    gradUs_norm.append(gradU/max_Wi)
    print(f"Maximum strain rate: {max_Wi}")

Maximum strain rate: 0.815914149465935
Maximum strain rate: 1.4525571678332805
Maximum strain rate: 2.6058001012084366
Maximum strain rate: 2.262828317639982
Maximum strain rate: 2.4281588656779762


In [20]:
gradUs_norm

[array([[[-0.45061825, -0.59479125,  0.08743619],
         [-0.3945834 ,  0.28403745, -1.7544757 ],
         [-0.6253586 ,  0.26136263,  0.1665808 ]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]]),
 array([[[-0.51956915, -0.16501923, -1.20036408],
         [-0.07201753,  0.93067316,  0.45167008],
         [-0.47758176, -0.10524652, -0.41110401]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ]]]),
 array([[[-0.53407954,  0.00616103,  0.39972988],
         [-0.85612377,  0.808279

In [22]:
for i, (time, gradU) in enumerate(zip(times, gradUs_norm)):
    np.savez_compressed(f"{output_file}_{i}", t=time, gradU=gradU)